In [25]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from pandas import Series, DataFrame
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [26]:
ds = pd.read_csv("C:/BDSE26/df_0907.csv",sep = ",")
ds

c:\Users\Student\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (25,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,sno,bike_lat,bike_lng,tot,time,date,Hr,weekday,weekday_num,holiday,...,Weather_lat,Weather_Dist,Generation,outlier,sarea,UVI,return_sec,rent_sec,rent_count,return_count
0,80,25.02751,121.495869,44,2017-01-04 12:00:00,2017-01-04,12,Wednesday,3,0,...,25.037658,2223.183703,1,True,萬華區,5,1729,15884,9,1
1,80,25.02751,121.495869,44,2017-01-07 05:00:00,2017-01-07,5,Saturday,6,1,...,25.037658,2223.183703,1,True,萬華區,0,1427,7507,2,1
2,80,25.02751,121.495869,44,2017-01-07 21:00:00,2017-01-07,21,Saturday,6,1,...,25.037658,2223.183703,1,True,萬華區,0,12290,16787,10,13
3,80,25.02751,121.495869,44,2017-01-09 07:00:00,2017-01-09,7,Monday,1,0,...,25.037658,2223.183703,1,True,萬華區,0,8997,8095,12,17
4,80,25.02751,121.495869,44,2017-01-14 12:00:00,2017-01-14,12,Saturday,6,1,...,25.037658,2223.183703,1,True,萬華區,1,5203,6130,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19212469,500112084,25.02976,121.564200,13,2022-04-30 17:00:00,2022-04-30,17,Saturday,6,1,...,25.037822,898.350579,2,True,信義區,0,2048,2914,3,2
19212470,500112084,25.02976,121.564200,13,2022-04-29 11:00:00,2022-04-29,11,Friday,5,0,...,25.037822,898.350579,2,True,信義區,6,741,645,2,1
19212471,500112084,25.02976,121.564200,13,2022-04-29 09:00:00,2022-04-29,9,Friday,5,0,...,25.037822,898.350579,2,True,信義區,4,0,2127,1,0
19212472,500112084,25.02976,121.564200,13,2022-04-29 18:00:00,2022-04-29,18,Friday,5,0,...,25.037822,898.350579,2,True,信義區,0,2402,3121,2,2


In [27]:
ds.columns

Index(['sno', 'bike_lat', 'bike_lng', 'tot', 'time', 'date', 'Hr', 'weekday',
       'weekday_num', 'holiday', 'school_off', 'Temperature', 'RH', 'WS',
       'rain', 'Precp', 'MRT_Out_ppl', 'ar', 'MRT_Dist', 'School_Dist', 'sna',
       'station', 'MRT_Station', 'MRT_No', 'MRT_Exit_Name', 'Exit_No',
       'MRT_lng', 'MRT_lat', 'School', 'School_lat', 'School_lng', 'StationID',
       'WeatherSiteID', 'WeatherSite', 'Weather_lng', 'Weather_lat',
       'Weather_Dist', 'Generation', 'outlier', 'sarea', 'UVI', 'return_sec',
       'rent_sec', 'rent_count', 'return_count'],
      dtype='object')

In [28]:
ds['sno'].astype('str')
ds['Hr'].astype('str')
ds['tot'].astype('str')
ds['weekday_num'].astype('str')

0           3
1           6
2           6
3           1
4           6
           ..
19212469    6
19212470    5
19212471    5
19212472    5
19212473    5
Name: weekday_num, Length: 19212474, dtype: object

In [29]:
#sno2 = ds[(ds.rent_count >= 100)]


In [30]:
# df[df.A > df.B]
#sno1 = ds[(ds.sno == 500105031)]


In [31]:
#sno2 = ds[(ds.date >= '2020-01-01')]

In [32]:
#ds['remaining_vehicles'] = ds['rent_count']-ds['return_count']

In [33]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X_choice_reindex = ds.reindex(columns=[
    'sno', 'tot', 'Hr', 'weekday_num', 'holiday', 'school_off', 'Temperature',
    'RH', 'WS', 'rain', 'Precp', 'MRT_Out_ppl', 'MRT_Dist', 'School_Dist',
    'Weather_Dist', 'Generation', 'return_sec', 'rent_sec', 'return_count'
])
X_choice_reindex

,sno,tot,Hr,weekday_num,holiday,school_off,Temperature,RH,WS,rain,Precp,MRT_Out_ppl,MRT_Dist,School_Dist,Weather_Dist,Generation,return_sec,rent_sec,return_count
0,80,44,12,3,0,0,25.5,66.0,4.2,0,0.0,1324,934.143832,118.041403,2223.183703,1,1729,15884,1
1,80,44,5,6,1,1,21.5,87.0,1.2,0,0.0,0,934.143832,118.041403,2223.183703,1,1427,7507,1
2,80,44,21,6,1,1,21.4,79.0,0.3,0,0.0,1556,934.143832,118.041403,2223.183703,1,12290,16787,13
3,80,44,7,1,0,0,18.1,69.0,3.0,0,0.0,1238,934.143832,118.041403,2223.183703,1,8997,8095,17
4,80,44,12,6,1,1,13.6,85.0,4.5,0,0.0,1687,934.143832,118.041403,2223.183703,1,5203,6130,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19212469,500112084,13,17,6,1,1,20.5,87.0,1.0,0,0.0,988,336.624961,396.446516,898.350579,2,2048,2914,2
19212470,500112084,13,11,5,0,0,30.6,61.0,1.9,0,0.0,896,336.624961,396.446516,898.350579,2,741,645,1
19212471,500112084,13,9,5,0,0,28.4,71.0,0.0,0,0.0,2736,336.624961,396.446516,898.350579,2,0,2127,0
19212472,500112084,13,18,5,0,0,21.2,87.0,1.9,0,0.0,1355,336.624961,396.446516,898.350579,2,2402,3121,2


In [34]:
# remaining_vehicles = X_choice_reindex[(X_choice_reindex.remaining_vehicles < 0)]
# remaining_vehicles

In [35]:
#from sklearn.preprocessing import MinMaxScaler
ds = X_choice_reindex.values
X = ds[:, 0:18]
y = ds[:, 18]
y = y.astype('int64')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
# from pyexpat import XML_PARAM_ENTITY_PARSING_ALWAYS
# from sklearn.preprocessing import StandardScaler
# X_StandardScaler= StandardScaler()
# X_train = X_StandardScaler.fit_transform(X_train)
# X_test = X_StandardScaler.fit_transform(X_test)


In [37]:
import xgboost as xgb
import pickle
#Scikit-learn 接口回歸模型
# 建立xgbrModel模型
xgbrModel=xgb.XGBRegressor(n_estimators=400,
                            learning_rate=0.1,
                            max_depth=7,
                            min_child_weight=3,
                            subsample=0.7,
                            colsample_bytree=0.9,
                            gamma=0.1,
                            reg_alpha=0.1,
                            reg_lambda=0.05,
                            silent=True,
                            tree_method='gpu_hist',
                            n_jobs=4)
# 使用訓練資料訓練模型
xgbrModel.fit(X_train,y_train)
kfold = StratifiedKFold(n_splits=10, random_state=1 ,shuffle=True)
results = cross_val_score(xgbrModel, X, y, cv=kfold)

pickle.dump(xgbrModel, open("XGBOOST回歸return.pkl", "wb"))

# print("Score: ",xgbrModel.score(X_train,y_train))
# print("Accuracy: %.2f%% (%.2f%%)" %
#       (results.mean() * 100, results.std() * 100))

[15:58:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




c:\Users\Student\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


[16:00:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:03:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:05:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false

In [ ]:
from sklearn import metrics
# 使用test資料預測
predicted = xgbrModel.predict(X_test)
#predicted = xgbrModel.predict(X_test)
print('R2 score: ', xgbrModel.score(X_test, y_test))
mse = metrics.mean_squared_error(y_test, predicted)
print('MSE score: ', mse)
# print("SCORE=", xgbrModel.score(X_test, y_test))
# print("MSE 訓練集->測試集", metrics.mean_squared_error(y_train, y_train_pred),"->", metrics.mean_squared_error(y_test, predicted))
# print("MAE 訓練集->測試集", metrics.mean_absolute_error(y_train, y_train_pred),"->", metrics.mean_absolute_error(y_test, predicted))


R2 score:  0.0
MSE score:  3.645653419962571e+25


In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,median_absolute_error,mean_absolute_percentage_error
#定義回歸模型評估誤差指標
print('回歸模型評估指標結果:')
print('均方誤差【MSE】:', mean_squared_error(y_test,predicted))
print('均方根誤差【RMSE】:',np.sqrt(mean_squared_error(y_test,predicted)))
print('平均絕對誤差【MAE】:',mean_absolute_error(y_test,predicted))
print('絕對誤差中位數【MedianAE】:',median_absolute_error(y_test,predicted))
print('平均絕對百分比誤差【MAPE】:',np.median(np.abs((predicted-y_test)/y_test)))
#print('絕對百分比誤差中位數【MedianAPE】:',median_absolute_percentage_error(y_test,predicted))

回歸模型評估指標結果:
均方誤差【MSE】: 2.9935482274071486e+25
均方根誤差【RMSE】: 5471332769451.287
平均絕對誤差【MAE】: 5467852939151.582
絕對誤差中位數【MedianAE】: 5497558138880.0
平均絕對百分比誤差【MAPE】: 5.960464477539062e-07


In [ ]:
xgbrModel.fit(X, y)
predicted = xgbrModel.predict(X)
ds['return_predicted'] = predicted
ds

[09:24:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




,sno,Hr,RH,MRT_Dist,School_Dist,tot,Temperature,Precp,MRT_Out_ppl,MRT_Dist,School_Dist,return,predicted
0,1,0,71.0,76.356469,382.304496,88,20.1,0.0,191,76.356469,382.304496,NaN,-9.223367e+18
1,1,1,73.0,76.356469,382.304496,88,19.9,0.0,0,76.356469,382.304496,NaN,-9.223367e+18
2,1,6,77.0,76.356469,382.304496,88,19.5,0.0,417,76.356469,382.304496,NaN,-9.223367e+18
3,1,7,75.0,76.356469,382.304496,88,19.6,0.0,772,76.356469,382.304496,NaN,-9.223367e+18
4,1,8,73.0,76.356469,382.304496,88,20.1,0.0,1197,76.356469,382.304496,NaN,-9.223367e+18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10870854,500119090,18,85.0,662.284970,155.193437,40,21.4,0.0,1268,662.284970,155.193437,NaN,-9.223366e+18
10870855,500119090,19,85.0,662.284970,155.193437,40,21.3,0.0,1013,662.284970,155.193437,NaN,-9.223366e+18
10870856,500119090,20,84.0,662.284970,155.193437,40,21.3,0.0,703,662.284970,155.193437,NaN,-9.223366e+18
10870857,500119090,21,82.0,662.284970,155.193437,40,21.4,0.0,761,662.284970,155.193437,NaN,-9.223366e+18


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X_choice_reindex1 = X_choice_reindex.reindex(columns=[
    'sno', 'tot', 'Hr', 'weekday_num', 'holiday', 'school_off', 'Temperature',
    'RH', 'WS', 'rain', 'Precp', 'MRT_Out_ppl', 'MRT_Dist', 'School_Dist',
    'Weather_Dist', 'Generation', 'return_sec', 'rent_sec', 'rent_count'
])
X_choice_reindex1

In [ ]:
ds = X_choice_reindex1.values
X = ds[:, 0:18]
y = ds[:, 18]
y = y.astype('int64')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
import xgboost as xgb
import pickle
#Scikit-learn 接口回歸模型
# 建立xgbrModel模型
xgbrModel=xgb.XGBRegressor(n_estimators=400,
                            learning_rate=0.1,
                            max_depth=7,
                            min_child_weight=3,
                            subsample=0.7,
                            colsample_bytree=0.9,
                            gamma=0.1,
                            reg_alpha=0.1,
                            reg_lambda=0.05,
                            silent=True,
                            tree_method='gpu_hist',
                            n_jobs=4)
# 使用訓練資料訓練模型
xgbrModel.fit(X_train,y_train)
kfold = StratifiedKFold(n_splits=10, random_state=1 ,shuffle=True)
results = cross_val_score(xgbrModel, X, y, cv=kfold)

pickle.dump(xgbrModel, open("XGBOOST回歸rent.pkl", "wb"))

In [ ]:
from sklearn import metrics
# 使用test資料預測
predicted = xgbrModel.predict(X_test)
#predicted = xgbrModel.predict(X_test)
print('R2 score: ', xgbrModel.score(X_test, y_test))
mse = metrics.mean_squared_error(y_test, predicted)
print('MSE score: ', mse)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,median_absolute_error,mean_absolute_percentage_error
#定義回歸模型評估誤差指標
print('回歸模型評估指標結果:')
print('均方誤差【MSE】:', mean_squared_error(y_test,predicted))
print('均方根誤差【RMSE】:',np.sqrt(mean_squared_error(y_test,predicted)))
print('平均絕對誤差【MAE】:',mean_absolute_error(y_test,predicted))
print('絕對誤差中位數【MedianAE】:',median_absolute_error(y_test,predicted))
print('平均絕對百分比誤差【MAPE】:',np.median(np.abs((predicted-y_test)/y_test)))
#print('絕對百分比誤差中位數【MedianAPE】:',median_absolute_percentage_error(y_test,predicted))

In [ ]:
xgbrModel.fit(X, y)
predicted = xgbrModel.predict(X)
ds['return_predicted'] = predicted
ds